In [ ]:
import numpy as np
import pandas as pd
import tqdm

import traceback
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from os_elm import OS_ELM


In [ ]:
# ===========================================
# Instantiate os-elm
# ===========================================
def initialiseOSELM(n_input_nodes, n_hidden_nodes, n_output_nodes):
    print("initialiseOSELM ===========================================")
    try:
        os_elm = OS_ELM(
            # the number of input nodes.
            n_input_nodes=n_input_nodes,
            # the number of output nodes.
            n_output_nodes=n_output_nodes,
            # the number of hidden nodes.
            n_hidden_nodes=n_hidden_nodes,
            
            # activation function sigmoid applied to the hidden nodes only
            activation='sigmoid',
        )
    except Exception as e:
        traceback.print_exc()
        raise e
    return os_elm

In [ ]:
# ****************************************
# Initialise Variables
# ****************************************
batch_size = 9
start_data_limit = 0
end_data_limit = batch_size
n_input_nodes = 3
n_hidden_nodes = 5
n_output_nodes = 5
n_classes = n_output_nodes
os_elm = initialiseOSELM(n_input_nodes, n_hidden_nodes, n_output_nodes)
x_classify_df = None
x_classify = None
evaluate_data = True
load_data = True
iteration = 1

print('restoring model parameters...') 
os_elm.restore('./checkpoint/model.ckpt')

while evaluate_data:
    print(f"start_data_limit : {start_data_limit}, end_data_limit : {end_data_limit}")
    # ===========================================
    # Load dataset
    # ===========================================
    if load_data:
        input_data = pd.read_csv("MultitestELM.csv")
                  
                
        pd.options.mode.chained_assignment = None
        Fwd_Header_Len_col = input_data['Fwd_Header_Len']
        Fwd_Header_Len_col.replace(to_replace = 0, value = Fwd_Header_Len_col.mean(), inplace=True)

        Fwd_IAT_Max_col = input_data['Fwd_IAT_Max']
        Fwd_IAT_Max_col.replace(to_replace = 0, value = Fwd_IAT_Max_col.mean(), inplace=True)

        input_Data_length = len(input_data.index)
        print(f'Total records to evaluate : {len(input_data.index)}')
        load_data = False

       
        df0 = input_data[input_data['Label'] == 0]
        print(f'Total records df0 : {len(df0.index)}')
        df1 = input_data[input_data['Label'] == 1]
        print(f'Total records df1 : {len(df1.index)}')
        df2 = input_data[input_data['Label'] == 2]
        print(f'Total records df2 : {len(df2.index)}')
        df3 = input_data[input_data['Label'] == 3]
        print(f'Total records df3 : {len(df3.index)}')
        df4 = input_data[input_data['Label'] == 4]
        print(f'Total records df4 : {len(df4.index)}')
        
        input_data = df0.append(df1, ignore_index=True).append(df2, ignore_index=True).append(df3, ignore_index=True).append(df4, ignore_index=True)
    
    x_classify_df_with_label = input_data[['Tot_Fwd_Pkts','Fwd_Header_Len','Fwd_IAT_Max', 'Label']]

    x_classify_df = x_classify_df_with_label[['Tot_Fwd_Pkts','Fwd_Header_Len','Fwd_IAT_Max']]
    print(f'Lenght of x_classify_df_with_label : {len(x_classify_df_with_label)} for iteration {iteration}')
    iteration = iteration + 1
    
    x_classify=np.array(x_classify_df[start_data_limit:end_data_limit])
    t_classify = x_classify_df_with_label['Label']
    t_classify=np.array(t_classify[:len(x_classify)])
    
    # if True:
    if end_data_limit > input_Data_length:
        print(f"end_data_limit : {end_data_limit}, input_Data_length : {input_Data_length}")
        evaluate_data = False

    start_data_limit = end_data_limit
    end_data_limit = end_data_limit + batch_size

    # ===========================================
    # Prepare dataset
    # ===========================================
    # shuffle dataset
    perm = np.random.permutation(len(x_classify))
    x_classify = x_classify[perm]
    t_classify = t_classify[perm]
    
    # min_max_scalar = MinMaxScaler().fit(x_classify)
    x_classifytest_normalised = MinMaxScaler().fit_transform(x_classify)
    t_classifytest_categorised = to_categorical(t_classify, num_classes=n_classes)

    print(f'x_train_normalised sahpe : {x_classifytest_normalised.shape}')
    print(f't_test_categorised sahpe : {t_classifytest_categorised.shape}')

    # ===========================================
    # ReEvaluation
    # ===========================================

    [precision, accuracy, score, recall] = os_elm.evaluate(x_classifytest_normalised, t_classifytest_categorised, metrics=['accuracy', 'precision', 'score', 'recall'])
    print(f'precision: {precision} , accuracy: {accuracy}, score: {score}, recall: {recall}')